In [1]:
"""
Feature: num_of_transfers
Count number of transfers per admission from transfers.csv
"""
import pandas as pd
import os
from config_paths import *
from utils import drop_columns

In [2]:
# Read dataset
df = pd.read_csv(dataset_path)
initial_row_count = len(df)

# Load transfers
transfers = pd.read_csv(
    os.path.join(hosp_path, "transfers.csv"),
    usecols=["subject_id", "hadm_id", "careunit"]
)

# Count transfers per admission (like notebook - uses "num_transfers")
loc_features = (
    transfers.groupby(["subject_id", "hadm_id"])
    .agg(num_transfers=("careunit", "count"))
    .reset_index()
)

# Merge with dataset
df = df.merge(
    loc_features,
    on=["subject_id", "hadm_id"],
    how="left"
)

# Fill missing values with 0 (notebook shows no missing values, but we'll fill for safety)
df["num_transfers"] = df["num_transfers"].fillna(0).astype(int)

# Also create num_of_transfers for consistency (if needed by other scripts)
df["num_of_transfers"] = df["num_transfers"]

# Save
df.to_csv(dataset_path, index=False)

# Diagnostic output
print(f"\nTotal rows: {len(df)}")
print(f"Expected total: {initial_row_count}")
if len(df) == initial_row_count:
    print(f"✓ Row count matches expected total ({initial_row_count})")
else:
    print(f"⚠ Row count mismatch! Expected {initial_row_count}, got {len(df)} (difference: {len(df) - initial_row_count})")
print(f"Dataset shape: {df.shape}")
print(f"Feature 'num_transfers' added. Range: {df['num_transfers'].min()} - {df['num_transfers'].max()}")
print(f"Dataset shape: {df.shape}")


Total rows: 158020
Expected total: 158020
✓ Row count matches expected total (158020)
Dataset shape: (158020, 52)
Feature 'num_transfers' added. Range: 2 - 34
Dataset shape: (158020, 52)


In [3]:
df.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes',
       'catheter_present', 'final_cauti_flag', 'final_insertion_date',
       'final_removal_date', 'catheter_duration_days', 'BMI_in_admission',
       'BMI_computed', 'BMI_last_year', 'BMI', 

In [5]:
# cols_to_drop = [
#  "charlson_score"
# ]
# df = drop_columns(df, cols_to_drop)
# df.to_csv(dataset_path, index=False)